In [7]:
# Load necessary libraries
library(reticulate)

# Set Python environment
use_python("C:/Users/Victory.Adogbeji/AppData/Local/Anaconda3/envs/tf_env/python.exe", required = TRUE)

# Import Python modules
tf <- import("tensorflow")
np <- import("numpy")
plt <- import("matplotlib.pyplot")

# Disable XLA JIT
tf$config$optimizer$set_jit(FALSE)

# Load and preprocess the data
fashion_mnist <- tf$keras$datasets$fashion_mnist
data <- fashion_mnist$load_data()

x_train <- data[[1]][[1]]
y_train <- data[[1]][[2]]
x_test <- data[[2]][[1]]
y_test <- data[[2]][[2]]

x_train <- x_train / 255.0
x_test <- x_test / 255.0

# Reshape with dimension check
x_train <- array_reshape(x_train, c(dim(x_train)[1], 28L, 28L, 1L))
x_test <- array_reshape(x_test, c(dim(x_test)[1], 28L, 28L, 1L))

# Verify dimensions
cat("x_test dimensions:", dim(x_test), "\n")

# Define class names
class_names <- c("T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot")

# Function to create CNN model
create_cnn_model <- function() {
  model <- tf$keras$models$Sequential(list(
    tf$keras$layers$Input(shape = as.integer(c(28, 28, 1))),
    tf$keras$layers$Conv2D(filters = 32L, kernel_size = as.integer(c(3, 3)), activation = "relu"),
    tf$keras$layers$MaxPooling2D(pool_size = as.integer(c(2, 2))),
    tf$keras$layers$Conv2D(filters = 64L, kernel_size = as.integer(c(3, 3)), activation = "relu"),
    tf$keras$layers$MaxPooling2D(pool_size = as.integer(c(2, 2))),
    tf$keras$layers$Flatten(),
    tf$keras$layers$Dense(units = 128L, activation = "relu"),
    tf$keras$layers$Dense(units = 10L, activation = "softmax")
  ))
  return(model)
}

# Compile and train the model
model <- create_cnn_model()
model$compile(
  optimizer = "adam",
  loss = "sparse_categorical_crossentropy",
  metrics = list("accuracy")
)

history <- model$fit(x_train, y_train, epochs = 10L, validation_split = 0.2, batch_size = 64L)

# Evaluate the model
eval_result <- model$evaluate(x_test, y_test)
cat(sprintf("Test accuracy: %.4f\n", eval_result[[2]]))

# Predict on a few samples and plot
sample_images <- x_test[1:2,,,, drop = FALSE]  # Simplified subsetting
sample_labels <- y_test[1:2]
predictions <- model$predict(sample_images)

# Verify sample_images dimensions
cat("sample_images dimensions:", dim(sample_images), "\n")

for (i in 1:2) {
  predicted_label <- which.max(predictions[i,]) - 1
  true_label <- sample_labels[i]
  
  cat(sprintf("Image %d:\n", i))
  cat(sprintf("Predicted: %s\n", class_names[predicted_label + 1]))
  cat(sprintf("True Label: %s\n", class_names[true_label + 1]))
  
  # Reshape and plot
  img_matrix <- array_reshape(sample_images[i,,,1], c(28L, 28L))  # Extract single channel
  plt$figure(figsize = as.integer(c(2, 2)))
  plt$imshow(img_matrix, cmap = "gray")
  plt$title(sprintf("Pred: %s, True: %s", class_names[predicted_label + 1], class_names[true_label + 1]))
  plt$axis("off")
  plt$savefig(sprintf("prediction_%d.png", i))
  plt$close()
}

# Save the model
model$save("fashion_mnist_cnn.keras")

x_test dimensions: 10000 28 28 1 
Test accuracy: 0.9084
sample_images dimensions: 2 28 28 1 
Image 1:
Predicted: Ankle boot
True Label: Ankle boot
Image 2:
Predicted: Pullover
True Label: Pullover
